### 获得单例文本预测结果

In [ ]:
from config import *
from utils import *

q_std_list, fst_q_std_sentence_embeddings, q_all, fst_q_all_sentence_embeddings_dict = read_q_std_q_corpus(q_std_file, fst_q_std_vectors_file, q_corpus_file, fst_q_corpus_vectors_file)
_, sec_q_std_sentence_embeddings, _, sec_q_all_sentence_embeddings_dict = read_q_std_q_corpus(q_std_file, sec_q_std_vectors_file, q_corpus_file, sec_q_corpus_vectors_file)

def get_fst_topK(text, K=10):
    text_embedding = model1.encode([text])[0].numpy()
    sims_with_std = np.array(cos_sim4matrix_2(text_embedding, fst_q_std_sentence_embeddings))
    sort_idx = np.argsort(-sims_with_std)[:K]
    sims_q_sort = [q_std_list[idx] for idx in sort_idx]
    sims_values = [sims_with_std[idx] for idx in sort_idx]
    result = list(zip(sims_q_sort, sims_values))
    return (result)

def get_sec_topK(self, text, K=20):
    text_embedding = self.model.encode([text])[0]
    sims_with_std = np.array(self.cos_sim4matrix_2(text_embedding, self.std_sentence_embeddings))
    sort_idx = np.argsort(-sims_with_std)[:K]
    sims_q_sort = [self.q_std_list[idx] for idx in sort_idx]
    sims_values = [sims_with_std[idx] for idx in sort_idx]
    result = list(zip(sims_q_sort, sims_values))
    return (result)

In [ ]:
import numpy as np
from config import *
from utils import *
from task_sentence_embedding_FinanceFAQ_step1_1 import model as model1
from task_sentence_embedding_FinanceFAQ_step2_1 import model as model2

text = input()

# 第一阶段召回
result_first = get_fst_topK(text=text)
print('第一阶段\n', result_first[0:20])
first_intents = [v[0] for v in result_first]

# 第二阶段召回
a_texts_embeddings_2 = np.array(model2.encode([text]))
b_texts_embeddings_2 = np.array([sec_q_all_sentence_embeddings_dict[v] for v in first_intents])
sims_with_std = cos_sim4matrix_2(a_texts_embeddings_2, b_texts_embeddings_2).reshape(-1)
sort_idx = np.argsort(-sims_with_std).tolist()
intents_sort = [first_intents[idx] for idx in sort_idx]
sims_values = [sims_with_std[idx] for idx in sort_idx]
result_second = list(zip(intents_sort, sims_values))
print('第二阶段\n', result_second[0:20])